In [61]:
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.neural_network import MLPRegressor
from random import randint, choice
from sklearn.preprocessing import StandardScaler

In [62]:
filename = 'Hackathon_Market research_keywords.xlsx'
activities = 'AD', 'DS', 'DP'
keywords = {}
for act in activities:
    data = pd.read_excel(filename, sheet_name=act)
    kw = []
    for col in ['keyword', 'synonyms', 'hyponyms']:
        kw_ = data[col].to_list()
        for kw__ in kw_:
            if not pd.isna(kw__):
                kw___ = [a.strip() for a in kw__.split(',')]
                kw.extend(kw___)
    keywords[act] = kw
KEYWORDS = []
for act in activities:
    KEYWORDS.extend(keywords[act])
N = len(KEYWORDS)
data_x = np.arange(N)[:, None]
data_y = np.zeros((N, 3))
s = 0
l = len(keywords[activities[0]])
data_y[s:l, :] = np.array([1,0,0])
s += l
l = len(keywords[activities[1]])
data_y[s:s+l, :] = np.array([0,1,0])
s += l
l = len(keywords[activities[2]])
data_y[s:s+l, :] = np.array([[0,0,1]])
N = len(KEYWORDS)
model = MLPRegressor(hidden_layer_sizes=(100,100))
sub = [randint(0, N-1) for k in range(1000)]
scaler_x = StandardScaler()
train_x = scaler_x.fit_transform(data_x[sub])
train_y = data_y[sub]
model.fit(train_x, train_y)
def predict(kwr):
    if kwr in KEYWORDS:
        idx = KEYWORDS.index(kwr)
        s = model.predict(scaler_x.transform([[idx]]))
        return s#np.argmax(s)
    else:
        return None
def norm(x):
    y = np.exp(x)
    return y/y.sum()

data = pd.read_csv('extracted_keywords.csv')
predictions = []
for i in range(len(data['domain'])):
    kws = eval(data.iloc[i]['keywords'])
    pred = np.array([0.0,0,0])
    s = 0
    for kw in kws:
        p = predict(kw)
        if not p is None:
            pred += p[0]
            s += 1
    pred /= s
    pred = norm(pred)
    pred_str = {'AD': pred[0], 'DS': pred[1], 'DP': pred[2]}
    predictions.append(str(pred_str))
data['predictions'] = predictions

In [63]:
DATA = data

In [64]:
out = widgets.Output(layout={'border': '1px solid black'})

In [65]:
params = [(k, v) for v, k in enumerate(DATA['domain'])]

In [66]:
wlist = widgets.SelectMultiple(
    options=[],
    value=[],
    rows=10,
    description='Keywords',
    disabled=False
)

In [67]:
def show_domain(x=params):
    data = eval(DATA.iloc[x]['predictions'])
    kwords = eval(DATA.iloc[x]['keywords'])
    wlist.options = kwords
    name = DATA.iloc[x]['company_name']
    text = list(data.keys())
    values = list(data.values())
    with out:
        out.clear_output()
        plt.cla() # Clearing the ax
        plt.bar(list(range(len(values))), values, edgecolor='k')
        for idx, t in enumerate(text):
            plt.text(idx, values[idx], f'{100*values[idx]:.0f}%', va='bottom', ha='center', fontsize = 16)
        plt.xticks(list(range(len(values))), [s.upper() for s in text], fontsize=16)
        plt.ylim(0, 0.8)
        plt.yticks([])
        plt.title(name)
        plt.show('off')
        
btn_approve = widgets.Button(
    description='Approve',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Approve!',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_reject = widgets.Button(
    description='Reject',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reject!',
    icon='ban' # (FontAwesome names without the `fa-` prefix)
)
def approve(*args):
    pass
def reject(*args):
    pass
btn_approve.on_click(approve)
btn_reject.on_click(reject)

In [68]:
w = widgets.interactive(show_domain)

In [69]:
w_predictions = widgets.VBox([w, widgets.HBox([out, widgets.VBox([wlist, widgets.HBox([btn_approve, btn_reject])])])])
w2 = widgets.VBox([out, w, btn_approve])

In [70]:
@widgets.interact()
def wid(Action=[('Predictions', 1), ('Domain blacklists', 2)]):
    if Action == 1:
        display(w_predictions)
    else:
        display(w_blacklist)

interactive(children=(Dropdown(description='Action', options=(('Predictions', 1), ('Domain blacklists', 2)), v…

In [72]:
# manual parse of the string into dictionary
text = DATA.iloc[0]['predictions']
tmp = {}
for s in text.replace('}', '').split(','):
    key = s.split("'")[1]
    val = float(s.split(':')[1].strip())
    tmp[key] = val

# Domains blacklist

In [73]:
domains_blacklist = pd.read_csv('domain_blacklist.txt', names=['domain'])['domain'].to_list()

In [74]:
#def remove_from_blacklist()

In [75]:
blist = widgets.SelectMultiple(
    options=domains_blacklist,
    value=[],
    rows=10,
    description='Domains',
    disabled=False
)
domain_add_to_blacklist = widgets.Text(
    value='',
    placeholder='...add domain...',
    description='',
    disabled=False
)

In [76]:
btn_add_to_blacklist = widgets.Button(
    description='Add to blacklist',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Add',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_remove_from_blacklist = widgets.Button(
    description='Remove from blacklist',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Remove',
    layout=widgets.Layout(width='95%', alignment='right'),
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
def add_to_blacklist(*args):
    new = domain_add_to_blacklist.value
    if len(new) > 0 and '.' in new:
        options = list(blist.options)
        if new not in options:
            options.append(new)
            options.sort()
            blist.options = options
def remove_from_blacklist(*args):
    selected = blist.value
    if len(selected) > 0:
        blist.options = [sel for sel in blist.options if sel not in selected]
            
btn_remove_from_blacklist.on_click(remove_from_blacklist)
btn_add_to_blacklist.on_click(add_to_blacklist)

In [77]:
w_blacklist = widgets.HBox([widgets.VBox([blist, btn_remove_from_blacklist]), widgets.VBox([domain_add_to_blacklist, btn_add_to_blacklist])], box_style='warning')

In [78]:
w_blacklist

HBox(box_style='warning', children=(VBox(children=(SelectMultiple(description='Domains', options=('*.acs.org',…

In [79]:
tab_contents = ['P0', 'P1', 'P2', 'P3', 'P4']
children = [w_predictions, w_blacklist]
tab = widgets.Tab()
tab.children = children
tab.titles = ('Company_activities', 'Domain blacklist', ) # does not work, wtf
tab